In [11]:
import requests
import psycopg2
from psycopg2 import sql
import os
from dotenv import load_dotenv
import pandas as pd
from Bitcoin_api import fetch_bitcoin_data
from new_scrappy import get_bitcoin_news

Python-dotenv could not parse statement starting at line 11


Fetched DataFrame:
             1. Open   2. High    3. Low  4. Close     5. Volume
Date                                                            
2024-06-06  71120.65  71200.00  71100.01  71133.30    178.275155
2024-06-05  70541.68  71804.13  70356.49  71121.11  10123.180292
2024-06-04  68791.04  71108.90  68542.37  70542.32  18042.193965
2024-06-03  67731.14  70295.78  67568.00  68791.00  11884.077311
2024-06-02  67719.29  68411.00  67260.00  67735.52   2997.152415
[Total Rows: 180]


In [18]:
print("Hello World!")

Hello World!


# BTC

## Connecting to DataBase and Creating Tables

In [12]:
# make sure to load dotenv
from dotenv import load_dotenv
import os
import psycopg2
load_dotenv()

# Database connection paramete(s
Database_URL = os.getenv('DataBase_Url')

#connect to database
conn = psycopg2.connect(Database_URL)
cur = conn.cursor()

#create tables if they don't exist

cur.execute('''
CREATE TABLE IF NOT EXISTS Bitcoin_Prices_Update(
    Date DATE PRIMARY KEY,
    Open FLOAT,
    High FLOAT,
    Low FLOAT,
    Close FLOAT,
    Volume FLOAT
    
)            
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS Bitcoin_News_Update(
    id SERIAL PRIMARY KEY,
    date DATE,
    title VARCHAR(500)
       
)            
''')

conn.commit()

Python-dotenv could not parse statement starting at line 11


## Insertion of Bitcoin Prices_Update to Table

In [13]:
#function to insert Bitcoin_Prices
def insert_bitcoin_data(conn, data):
    try:
        cursor = conn.cursor()
        for index, row in data.iterrows():
            cursor.execute('''
                INSERT INTO Bitcoin_Prices_Update(Date, Open, High, Low, Close, Volume)
                Values(%s, %s, %s, %s, %s, %s)
                ON CONFLICT (Date) DO NOTHING
                ''',(index, row['1. Open'], row['2. High'], row['3. Low'], row['4. Close'], row['5. Volume']))
        conn.commit()
        cursor.close()
        print("success")
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

## Insertion of  Bitcoin News_ Update to Table

In [8]:
#function to insert Bitcoin_News
def insert_bitcoin_news(conn, df):
    try:
        cursor = conn.cursor()
        for index, row in df.iterrows():
            cursor.execute('''
                INSERT INTO Bitcoin_News_Update (id, date, title)
                Values(%s, %s, %s)
                ON CONFLICT (id) DO NOTHING
                ''',(index,row['date'], row['title']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

## Fetching and Inserting Data

In [14]:
#Main Function to orchestrate the data fetching and insertion
def main():
    # fetch Bitcoin Data
    bitcoin_data = fetch_bitcoin_data("BTC","USD")
    #Scrape Bitcoin News data
    #bitcoin_titles_df = get_bitcoin_news()
    #connect to postgresSQL database
    conn = psycopg2.connect(Database_URL)

    if conn:
        #Insert data into database
        insert_bitcoin_data(conn,bitcoin_data)
        #insert_bitcoin_news(conn,bitcoin_titles_df)

        #close the database connection
        conn.close()
    else:
        print('Failed to connect to the database')

if __name__== '__main__':
    main()

Python-dotenv could not parse statement starting at line 11


success


#### Used for dropping tables

In [10]:
import os
import psycopg2

def drop_tables(table_names, conn):
    """
    Drops the specified tables from the PostgreSQL database.

    Args:
    table_names (list of str): List of table names to be dropped.
    conn: The psycopg2 connection object to the PostgreSQL database.
    """
    # Create a cursor object
    cur = conn.cursor()

    # Iterate over the list of table names and drop each one
    for table_name in table_names:
        drop_table_query = f'DROP TABLE IF EXISTS {table_name} CASCADE'
        cur.execute(drop_table_query)
        print(f'Table {table_name} dropped.')

    # Commit the changes and close the cursor
    conn.commit()
    cur.close()

if __name__ == "__main__":
    # List of table names to be dropped
    table_names = [
        'bitcoin_prices_update'
        
        
        
    ]

    # Connect to the PostgreSQL database using the DATABASE_URL environment variable
    DATABASE_URL = os.getenv('DATABASE_URL')
    conn = psycopg2.connect(DATABASE_URL)

    try:
        # Call the function to drop the tables
        drop_tables(table_names, conn)
    finally:
        if conn:
            conn.close()


Table bitcoin_prices_update dropped.
